In [ ]:
import torch

torch.Size([12])
<built-in method numel of Tensor object at 0x000001C257C1A5C0>
reshaped X: tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


### 1. 常用的torch创建方式

In [5]:
# 创建元素从0到12的张量
x = torch.arange(end=12)
print(x.shape)
print(x.numel)

torch.Size([12])
<built-in method numel of Tensor object at 0x000001C257CAD120>


In [6]:
# reshape修改维度
X = x.reshape(3,4)
print(f"reshaped X: {X}")

reshaped X: tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


In [7]:
# 创建全为0或1的张量
ones = torch.ones(2,3,4)
zeros = torch.zeros(2,3,4)
print(ones)
print(zeros)

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


### 2. 广播机制
形状不同的张量也可以相加，导致结果错误\
a = (0,1,2), b = (0,1,2)$^\top$\
a+b=$\begin{bmatrix}
  0&  1& 2\\
  0&  1& 2\\
  0&  1& 2
\end{bmatrix}+
\begin{bmatrix}
  0& 0 & 0\\
  1& 1 & 1\\
  2&  2&2
\end{bmatrix}=
\begin{bmatrix}
  0& 1 & 2\\
  1& 2 & 3\\
  2& 3 & 4
\end{bmatrix}$

In [8]:
a = torch.arange(3).reshape(1,3)
b = torch.arange(3).reshape(3,1)
a+b

tensor([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4]])

In [12]:
# 切片机制
x_ = torch.ones((3, 4))
x_[1:3,::2]

tensor([[1., 1.],
        [1., 1.]])

### 3. tensor转numpy

In [14]:
a = torch.arange(3)
b = a.numpy()
print(type(a), type(b))

<class 'torch.Tensor'> <class 'numpy.ndarray'>


In [17]:
a = torch.arange(27).reshape((3,3,3))
a[0,:,:] = 1
a[1,:,:] = 2
a[2,:,:] = 3
# 固定后两个维度求和
print(a.sum(axis=[1,2]))

# 固定前两个梯度求和
print(a.sum(axis=[0,1]))

tensor([ 9, 18, 27])
tensor([18, 18, 18])


### 4. backward()求导数
1. 对于y=f(x), 创建x时需要x.requires_grad_(True) 或 在定义中给定参数 requires_grad=True
2. 将x输入到y之后，使用y.backward()对x求导
3. x.grad.zero()将x的梯度置为0

In [18]:
def gradient(x):
    y = 2 * torch.dot(x, x)
    y.backward()
    return x.grad

x = torch.arange(4.0).requires_grad_(True)
a = gradient(x)
print(a)
# 求导后将导数置0
x.grad.zero_()
print(a)

tensor([ 0.,  4.,  8., 12.])
tensor([0., 0., 0., 0.])


### 5. 张量复制（分配新内存）

In [19]:
A = torch.arange(20, dtype=torch.float32).reshape((5,4))
B = A.clone()
A * 2 == A + B

tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

张量求和操作
1. keepdims=True参数保持某一维度不求和
2. 利用广播机制求和,A / A.sum
3. 某个轴计算A的元素的累计总和

In [21]:
A = torch.arange(20, dtype=torch.float32).reshape((5,4))
A_sum = A.sum(axis=0, keepdims=True) # axis = 0 使得A的行保持不求和,对列求和
A_sum1 = A.cumsum(axis=0)
print(A_sum)
print(A / A_sum)
print(A_sum1)

tensor([[40., 45., 50., 55.]])
tensor([[0.0000, 0.0222, 0.0400, 0.0545],
        [0.1000, 0.1111, 0.1200, 0.1273],
        [0.2000, 0.2000, 0.2000, 0.2000],
        [0.3000, 0.2889, 0.2800, 0.2727],
        [0.4000, 0.3778, 0.3600, 0.3455]])
tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  6.,  8., 10.],
        [12., 15., 18., 21.],
        [24., 28., 32., 36.],
        [40., 45., 50., 55.]])
